# setup

In [ ]:
#setup requirements
%pip install -r ./../requirements.txt

In [ ]:
# load openai_api_key from .env file
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

model="gpt-4"


In [ ]:
# import of langchain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

# copy-paste Medienmitteilung

In [ ]:
# copy-paste a media release as 'text_input_medienmitteilung'
text_input_medienmitteilung = """

    # Medienmitteilung

    ## Horizon-Paket: Bund finanziert weitere Übergangsmassnahmen für Einzelprojekte
    
    Bern, 25.10.2023 - Der Bund stellt für die Starting und Consolidator Grants 2024 des Europäischen Forschungsrates (European Research Council, ERC) Mittel für Übergangsmassnahmen zur Verfügung. Dies hat der Bundesrat am 25. Oktober 2023 beschlossen. Aufgrund der aktuellen Nicht-Assoziierung am Horizon-Paket sind Forschende in der Schweiz zu den entsprechenden europäischen Ausschreibungen nicht zugelassen.

    Die Grants des Europäischen Forschungsrates fördern exzellente Grundlagenforschung und sind ein zentrales Förderinstrument für Spitzenforschende in der Schweiz. Diese waren bei vergangenen Ausschreibungen im internationalen Vergleich überdurchschnittlich erfolgreich. Daher soll Forschenden in der Schweiz, wie bereits in den Vorjahren, ein adäquater nationaler Ersatz angeboten werden. Der Bundesrat stellt dafür maximal 84 Millionen Franken zur Verfügung. Dafür werden Mittel eingesetzt, die das Parlament Ende 2020 für die Teilnahme der Schweiz am Horizon-Paket gesprochen hatte. Das Staatssekretariat für Bildung, Forschung und Innovation (SBFI) wird den Schweizerischen Nationalfonds mit den Ausschreibungen «SNSF Starting Grants 2024» und gegebenenfalls «SNSF Consolidator Grants 2024» beauftragen.

    Der Bundesrat verfolgt weiterhin das Ziel einer raschestmöglichen Assoziierung der Schweiz am Horizon-Paket. Die Ausschreibungen 2024 für die europäischen Stipendien (ERC Starting Grants, ERC Consolidator Grants) schliessen jedoch bereits im Oktober bzw. Dezember 2023. Daher sind sie den Forschenden in der Schweiz auch im Falle einer Assoziierung im Jahr 2024 nicht zugänglich.

    Horizon Europe

    Horizon Europe, das neunte EU-Rahmenprogramm für Forschung und Innovation, läuft von 2021 bis 2027 und ist mit einem Budget von gut 95 Milliarden Euro das weltweit grösste Forschungs- und Innovationsförderprogramm. Die Schweiz war am Vorgängerprogramm, Horizon 2020, assoziiert. Der Bundesrat strebt raschestmöglich den gleichen Status für Horizon Europe und damit verbundene Programme und Initiativen an (Euratom-Programm, ITER und Digital Europe Programme). Der Bundesrat hat für die Ausschreibungen 2021, 2022 und 2023 bereits Übergangsmassnahmen in der Höhe von 1,85 Milliarden Schweizer Franken zur Verfügung gestellt.

    Adresse für Rückfragen

    Kommunikationsdienst GS-WBF
"""

# run this ai-machine

## Step 1:
important facts

In [ ]:
# result_facts: Analyze the following text and identify all important facts
prompt = PromptTemplate.from_template("""
Analyze the following text and 
    identify all important facts.  
    The text is from news.admin.ch and is an 
    official media release: \n {medienmitteilung}\n\n
    rule: answer in same language as the media release!
    Your fact analysis:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_facts = runnable.invoke({"medienmitteilung": text_input_medienmitteilung})
print(result_facts)

## step 2:
keywords

In [ ]:
# result_top10keywords: Identify 10 keywords from the facts
prompt = PromptTemplate.from_template("""
    Identify 10 keywords that are likely to rank well 
    on Google (Switzerland). 
    The keywords are in the same language as the 
    following text and start with a #.

    Text: 
    {facts}
    
    Examples: #FederalCouncil #Geneva #Nyon #Motorway #A1 #Traffic #Trafficflow #Motorway     
    Your keywords incl. #:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_top10keywords = runnable.invoke({"facts": result_facts})
print(result_top10keywords)

In [ ]:
# result_top3keywords: Analyze the following keywords and text. 
prompt = PromptTemplate.from_template("""
    Analyze the following keywords and text. 
    Identify exactly 3 keywords that can rank 
    well on Google (Switzerland). 

    Examples: #FederalCouncil #ChuckNorris #Thailand  (on one line)
   
    ===============
    Keywords: 
    {top10keywords}

    Text: 
    {medienmitteilung}
    ===============
    Your top 3 keywords in the same language as the text/keywords:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_top3keywords = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top10keywords": result_top10keywords})
print(result_top3keywords)

## step 3:
social media content


In [ ]:
# result_tweets: write 3 matching tweets for a specific media release, facts, keywords.
prompt = PromptTemplate.from_template("""
    You are a neutral media spokesperson for the Federal Council 
    and write 3 matching tweets for a specific media release.
    
    Your tweets always contain:
    1. the 2 most important facts
    2. the 3 most important keywords
    3. 4 matching emojis.
    4. the tweet ends with the string [url].

    Rules:
    You answer in the same language as the media release.

    Sources: 
    ===========
    Facts: {facts}
    Top-Keywords: {top3keywords}
    Media release: {medienmitteilung}
    ==============
    Your 3 different tweet suggestions:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_tweets = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_tweets)

## step 4:
prepare for questions

In [ ]:
# result_fragen_20min: from 20min.ch/Wagenknecht and will collect 3 critical questions. 
prompt = PromptTemplate.from_template("""
    You are a reporter (male) for 20min.ch and 
    will collect 10 critical questions. 
    At the end we need the 3 most critical 
    questions.

    You think like Sahra Wagenknecht. 

    It is important that the questions 
    directly to the press release.
    
    Important:
    The questions are critical but not offensive. 
    and refer to the media release, the facts and a top keyword.

    Rules:
    Answer in the same language as the media release.

    Sources: 
    ===========
    Facts: {facts}
    Top-Keywords: {top3keywords}
    Media release: {medienmitteilung}
    ==============
    Your top-3 moust critical questions:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_fragen_20min = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_fragen_20min)

In [ ]:
# result_fragen_NZZ: from NZZ/Donald Duck and will collect 3 critical questions. 
prompt = PromptTemplate.from_template("""
You are a reporter (female) for the NZZ and will 
    collect 10 critical questions. 
    At the end we need the 3 most critical 
    questions.

    You think like Donald Duck.

    It is important that the questions 
    relate directly to the press release.
    
    This is important:
    The questions are critical, but not insulting. 
    And refer to the press release, the facts and a keyword.

    Rules:
    Answer in the same language as in the press release.

    Sources: 
    ===========
    Facts {facts}
    Keywords: {top3keywords}
    Media Release: {medienmitteilung}
    ==============
    Your top 3 most critical questions:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_fragen_nzz = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_fragen_nzz)

## step 5:
explain to a child

In [ ]:
# result_zusammenfassung_kind: summary for a 10 year old child 
prompt = PromptTemplate.from_template("""
Create a summary of the following text 
    for a 10 year old child of a media release.

    Important rule:
    Answer in the same language as the media release!

    Sources: 
    ===========
    Facts: {facts}
    Media release: {medienmitteilung}
    ==============
    Your child-friendly summary in correct language:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_zusammenfassung_kind =  runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "facts": result_facts})
print(result_zusammenfassung_kind)

## step 6:
illustrate

In [ ]:
# result_dalle3prompt: DALL-E prompt : black and white comic book panel in style of asterix & obelix in mid-21th 
# copy-paste the result_dalle3prompt into a good image generator like Midjourney or DallE-3 
prompt = PromptTemplate.from_template("""
Create a prompt to generate 
    with DALL-E a matching image.
    the prompt must be child-friendly.

    Important Roules:
    =================
    1) Your DALL-E Prompt must be in english.
    2) It must include the following texts:
        A)"create a black and white comic book panel"
        B)"comic illustration reminiscent of European adventure comics of the mid-21th century"
        C)"in a informative way." 
        D) [SCENE]
        E) The scene plays in switzerland.
        F) Prompt must be length 1000 or less.
    3) "Include a dramatic sound effect text saying &quot;{top3keywords}&quot;
    =================

    Contextinformation for the [SCENE]: {zusammenfassung_kind}
    ==============
    Generated the [SCENE] and following the rules 1-3!
    ==============
    Your DALL-E Prompt follows rules 1-3:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_dalle3prompt =  runnable.invoke({"zusammenfassung_kind": result_zusammenfassung_kind, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_dalle3prompt)

In [ ]:
# link to a DELL-E 2 image (Midjourney or DallE-3 are much better!)
from langchain.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="{image_desc}",
)
chain = LLMChain(llm=llm, prompt=prompt)
image_url = DallEAPIWrapper().run(chain.run(result_dalle3prompt))
image_url